In [1]:
import os
prod_id = 'AIAAVPU' #'AIAAVF'
directory_path = os.path.normpath(os.path.join(os.getcwd(),"..","data","AIA_AVPU")) #os.path.normpath(os.path.join(os.path.abspath(__file__),"..","..","data","AIA_AVPU"))
print(directory_path)
idx_name = 'aia_avpu' #'aia_avf'  #os.environ["AZURE_SEARCH_LANGCHAIN_RECURSIVETEXTSPLITTER_INDEX"]
file_id = 'AIA_Voluntary_Health_Insurance_Privilege_Ultra'

/mnt/d/BT/SRC/MS/DEMO/WrApp/Rag/data/AIA_AVPU


# Load MD

In [2]:
titleLst = []
chunkLst = []

import os  
  
def read_files_line_by_line(directory_path):  
    try:  
        # Get the list of all files and directories in the specified directory  
        files = os.listdir(directory_path)  
  
        # Iterate over the list and read file contents line by line  
        for file in files:  
            file_path = os.path.join(directory_path, file)  
            if os.path.isfile(file_path):  
                with open(file_path, 'r') as f:  
                    print(f"Contents of {file}:")  
                    lnIdx = 0
                    ctnt = ''
                    for line in f:
                        if lnIdx == 0 and line.startswith('. . . '):
                            break
                        if lnIdx == 0:
                            titleLst.append(line.strip())
                        elif line.startswith('<!--'):
                            continue #@# REFACTOR 还可过滤掉figure,过小的chunk etc
                        else:
                            ctnt = ctnt + line
                        print(line, end='')  # Print each line without adding extra newline  
                        lnIdx += 1
                    if not (ctnt=='' or ctnt.isspace()):
                        chunkLst.append(ctnt)
                    print("\n" + "-" * 40)  # Separator for readability  
  
    except FileNotFoundError:  
        print("The specified directory does not exist.")  
    except Exception as e:  
        print(f"An error occurred: {e}")  
  
  
# Call the function  
read_files_line_by_line(directory_path)  

Contents of AIAAVPU-00.md:

----------------------------------------
Contents of AIAAVPU-01.md:
AIA VOLUNTARY HEALTH INSURANCE PRIVILEGE ULTRA SCHEME (AVPU). . . 

Privileged medical protection bolsters your health shield  
友扣稅
TAX DEDUCTIBLE  
AIA Vitality  
View e-copy  
<figure>  
AIA  
HEALTHIER, LONGER,
BETTER LIVES  
</figure>  
HIGHER AND WIDER
PROTECTION AGAINST
MEDICAL INFLATION
GIVE YOU A BOOST OF ASSURANCE FOR THE NEW
CHALLENGES IN HEALTH THREATS AND ACCESS TO
APPROPRIATE HIGH-QUALITY TREATMENT  
Health threats continue to evolve. Quality healthcare services are becoming much
sought-after, especially in the face of increasing prevalence of cancer, and the rising
cost of medical expenses and innovative treatments. By having a wide range of medical
protection, it gives you access to medical treatment quickly and conveniently.  
We understand the challenges you are facing ...  
<figure>
</figure>  
<figure>
</figure>  
<table>
<tr>
<th>Medical inflation</th>
<th></th>
<th colsp

# Upload to Azure AI Search

## Load .env file (Copy .env-sample to .env and update accordingly)

In [3]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv('../../../.env') # take environment variables from .env.

# Variables not used here do not need to be updated in your .env file
search_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT_4AISEARCH"]
azure_openai_embedding_deployment_id = "text-embedding-3-small"#@# REFACTOR 可以用其它的emb或ranker,基于CLS的或非CLS但是多路的

search_credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"]) if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else DefaultAzureCredential()
azure_openai_key = os.environ["AZURE_OPENAI_API_KEY_4AISEARCH"] if len(os.environ["AZURE_OPENAI_API_KEY_4AISEARCH"]) > 0 else None
azure_openai_embedding_dimensions = 1024

print(search_endpoint)


https://estusaisrch.search.windows.net


## Setup Clinet

In [8]:
from openai import AzureOpenAI
from azure.identity import get_bearer_token_provider

azure_openai_client = None
# if azure_openai_key:
azure_openai_client = AzureOpenAI(
    api_key=azure_openai_key, 
    api_version="2023-05-15",
    azure_deployment=azure_openai_embedding_deployment_id,
    azure_endpoint=azure_openai_endpoint)
# else:
#     azure_openai_client = AzureOpenAI(
#         azure_ad_token_provider=get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"),
#         api_version="2023-05-15",
#         azure_deployment=azure_openai_embedding_deployment_id,
#         azure_endpoint=azure_openai_endpoint)

## Create Index

In [9]:
from azure.search.documents.indexes import SearchIndexClient

from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataSourceConnection,
    SearchIndexerDataContainer,
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration,
    AzureOpenAIEmbeddingSkill,
    SplitSkill,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
)
# Required to use the preview SDK
from azure.search.documents.indexes._generated.models import (
    SearchIndexerSkillset,
    AzureOpenAIVectorizer,
    AzureOpenAIParameters,
    SearchIndexerIndexProjections,
    SearchIndexerIndexProjectionSelector,
    SearchIndexerIndexProjectionsParameters,
    InputFieldMappingEntry,
    OutputFieldMappingEntry
)

def create_search_index(index_name, azure_openai_endpoint, azure_openai_embedding_deployment_id, azure_openai_key=None):
    return SearchIndex(
        name=index_name,
        fields=[
            SearchField(
                name="chunk_id",
                type=SearchFieldDataType.String,
                key=True,
                hidden=False,
                filterable=True,
                sortable=True,
                facetable=False,
                searchable=True,
                # analyzer_name="keyword"
            ),
            SearchField(
                name="prod_id",
                type=SearchFieldDataType.String,
                hidden=False,
                filterable=True,
                sortable=False,
                facetable=False,
                searchable=True
            ),
            SearchField(
                name="file_id",
                type=SearchFieldDataType.String,
                hidden=False,
                filterable=True,
                sortable=False,
                facetable=False,
                searchable=True
            ),
            SearchField(
                name="chunk",
                type=SearchFieldDataType.String,
                hidden=False,
                filterable=False,
                sortable=False,
                facetable=False,
                searchable=True
            ),
            SearchField(
                name="title",
                type=SearchFieldDataType.String,
                hidden=False,
                filterable=False,
                sortable=False,
                facetable=False,
                searchable=True
            ),
            SearchField(
                name="chunkVector",
                type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                hidden=False,
                filterable=False,
                sortable=False,
                facetable=False,
                searchable=True,
                vector_search_dimensions=azure_openai_embedding_dimensions,
                vector_search_profile_name="vecprofile"
            ),
            SearchField(
                name="titleVector",
                type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                hidden=False,
                filterable=False,
                sortable=False,
                facetable=False,
                searchable=True,
                vector_search_dimensions=azure_openai_embedding_dimensions,
                vector_search_profile_name="vecprofile"
            )
        ],
        vector_search=VectorSearch(
            profiles=[
                VectorSearchProfile(
                    name="vecprofile",
                    algorithm_configuration_name="hnsw-algorithm",
                    vectorizer="azure-openai-vectorizer"
                )
            ],
            algorithms=[
                HnswAlgorithmConfiguration(name="hnsw-algorithm")
            ],
            vectorizers=[
                AzureOpenAIVectorizer(
                        name="azure-openai-vectorizer",
                        azure_open_ai_parameters=AzureOpenAIParameters(
                            resource_uri=azure_openai_endpoint,
                            deployment_id=azure_openai_embedding_deployment_id,
                            model_name=azure_openai_embedding_deployment_id,
                            api_key=azure_openai_key # Optional if using RBAC authentication
                        )
                    )
            ]
        ),
        semantic_search=SemanticSearch(configurations=[
            SemanticConfiguration(
                name="my-semantic-config",
                prioritized_fields=SemanticPrioritizedFields(
                    title_field=SemanticField(field_name="title"),
                    # keywords_fields=[SemanticField(field_name="category")],
                    content_fields=[SemanticField(field_name="chunk")]
                )
            )
        ])   
    )


search_index_client = SearchIndexClient(endpoint=search_endpoint, credential=search_credential)
rts_searchindex = create_search_index(
    idx_name,
    azure_openai_endpoint,
    azure_openai_embedding_deployment_id,
    azure_openai_key
)
search_index_client.create_or_update_index(rts_searchindex)

print("Created recursive text splitter index")


Created recursive text splitter index


## Embed Chunks

In [10]:


title_response = azure_openai_client.embeddings.create(input=titleLst, model=azure_openai_embedding_deployment_id, dimensions=azure_openai_embedding_dimensions)
title_embeddings = [item.embedding for item in title_response.data]
content_response = azure_openai_client.embeddings.create(input=chunkLst, model=azure_openai_embedding_deployment_id, dimensions=azure_openai_embedding_dimensions)
content_embeddings = [item.embedding for item in content_response.data]


## Upload chunks to search index

In [11]:
recursive_search_client = search_index_client.get_search_client(idx_name)


docs2idx = [
    {
        "file_id": file_id,
        "chunk_id": f"{file_id}_{i}",
        "prod_id": prod_id,
        "chunk": chunk,
        "title": titleLst[i],
        "titleVector": title_embeddings[i],
        "chunkVector": content_embeddings[i],
    }
    for i, chunk in enumerate(chunkLst)
]

recursive_search_client.upload_documents(docs2idx)

print("Uploaded chunks and embeddings for recursive text splitter")

Uploaded chunks and embeddings for recursive text splitter


# Weighted Search

In [4]:
from azure.search.documents import SearchClient
search_client = SearchClient(endpoint=search_endpoint, index_name=idx_name, credential=search_credential)

from azure.search.documents.models import VectorizableTextQuery
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType

In [14]:
# Multi-Vector Search
query = """3. **Children's Needs:**
   - The user has two children, aged 7 and 5, which may imply a need for family health coverage. """
  
for i in range(4):
    vector_query_1 = VectorizableTextQuery(text=query, k_nearest_neighbors=1, fields="titleVector", weight=1)
    vector_query_2 = VectorizableTextQuery(text=query, k_nearest_neighbors=1, fields="chunkVector", weight=1)

    results = search_client.search(  
        search_text=None, #query,  
        vector_queries=[vector_query_1, vector_query_2],
        select=["title", "chunk", "chunk_id"],
        # query_type=QueryType.SEMANTIC, semantic_configuration_name='my-semantic-config', query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
        top=2
    )  
      
    for result in results:  
        print(f"Title: {result['title']}")  
        print(f"Score: {result['@search.score']}")  
        print(f"Content: {result['chunk']}")  
        print(f"CHUNK ID: {result['chunk_id']}")  
        # print(f"Reranker Score: {result['@search.reranker_score']}")
        # captions = result["@search.captions"]
        # if captions:
        #     caption = captions[0]
        #     if caption.highlights:
        #         print(f"Caption: {caption.highlights}")
        #     else:
        #         print(f"Caption: {caption.text}")
        print("\n")


Title: Example. . .
Score: 0.01666666753590107
Content: 
(The following example is hypothetical and for illustrative purposes only. If there are any changes in the
values, no separate announcement will be made. It does not constitute any medical advice. You should
seek independent professional advice before making any decision on this matter.)  
<table>
<tr>
<td>Policyowner and insured person:</td>
<td>Jack (age 40, non-smoker)</td>
</tr>
<tr>
<td>Occupation:</td>
<td>Business Development Director</td>
</tr>
<tr>
<td>Family status:</td>
<td>Married, with a son</td>
</tr>
<tr>
<td>Current cover:</td>
<td>Employer's group medical plan</td>
</tr>
</table>  
<figure>
</figure>  
As a successful business executive, Jack wants himself and his loved ones to have access to advanced and
flexible medical protection. He decides to purchase three AIA Voluntary Health Insurance Privilege Ultra
Scheme policies with worldwide (excluding United States) geographical cover and HKD16,000/USD2,000
annual 